In [ ]:
#!pip -q install transformers

In [ ]:
#!pip -q install bertviz


## 1- Scaled dot-product attention

In [ ]:
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

In [ ]:
model_ckpt = "bert-base-uncased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
model = BertModel.from_pretrained(model_ckpt)

In [ ]:
text = "time flies like an arrow"

In [ ]:
show(model, "bert", tokenizer, text, display_mode="dark", layer=0, head=8)

###Question 
What can you see from the visualisation ? Can you expalain that ?
### Answer
*   The values of the query and key vectors are represented as vertical bands
*   The intensity of each band corresponds to the magnitude.
*   The connecting lines are weighted according to the attention between the tokens
*   The query vector for “flies” has the strongest overlap with the key vector for “arrow”.

## 2- Understanding Queries, Keys, and Values

### Q = What is the intuition behind using the Keys, and Values, Query ?  can you give an exampe ? 
A = The notion of query, key, and value vectors may seem a bit difficult to understand the first time you encounter them. 
Imagine that you’re at the supermarket buying all the ingredients you need for your dinner. You have the dish’s recipe, and each of the required ingredients can be thought of as a **query**. 
As you scan the shelves, you look at the labels (**keys**) and check whether they match an ingredient on your list (**similarity function**). If you have a match, then you take the item (**value**) from the shelf. 


In [ ]:
# try to tokenize the text, and extract the input IDs
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids


In [ ]:
## try to get the embedding 
from torch import nn
from transformers import AutoConfig
config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

In [ ]:
## How can you get the embedding for your input sentence ? what is the size of your representation
#[batch_size, seq_len, hidden_dim]
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds.size()


In [ ]:
#create the query, key, and value vectors and calculate the attention scores 
#using the dot product as the similarity function:

import torch
from math import sqrt
query = key = value = inputs_embeds

In [ ]:
## How can you get the dimesion of the key ?  
## refer to the slides
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k)
scores.size()
## created a 5×5 matrix of attention scores per sample in the batch

In [ ]:
## How can you apply  apply the softmax to these matrices
import torch.nn.functional as F

weights = F.softmax(scores, dim=-1)
weights.sum(dim=-1)

In [ ]:
## multiply the attention weights by the values
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

In [ ]:
## write a function to calculate the self attention

def scaled_dot_product_attention(query, key, value):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

In [ ]:
#plot y=sin(2/10000^(2*x/512))
